In [1]:
import os
import glob
import numpy as np
from pyproj import Transformer
from scipy.spatial.transform import Rotation as R

In [2]:
out_fol = "/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/ID-MCS/20240420/proc_out"
im_list = sorted(glob.glob(os.path.join(out_fol, '..', 'SkySatScene_TOAR', '*_analytic.tif')))
cam_list = sorted(glob.glob(os.path.join(out_fol, 'cam_gen', '*.tsai')))
gcp_list = sorted(glob.glob(os.path.join(out_fol, 'cam_gen', '*.gcp')))
cam_gen_fol = os.path.join(out_fol, 'cam_gen')

print(f'Found {len(im_list)} images')
print(f'Found {len(cam_list)} cameras')
print(f'Found {len(gcp_list)} GCPs')


Found 76 images
Found 76 cameras
Found 76 GCPs


In [ ]:
### CONVERT TSAI FILES FROM ASP TO A SINGLE TXT FILE FOR METASHAPE IMPORT ###

def parse_tsai(tsai_path):
    with open(tsai_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    C_line = next(l for l in lines if l.startswith('C ='))
    R_line = next(l for l in lines if l.startswith('R ='))

    C = np.fromstring(C_line.split('=')[1], sep=' ')
    R_flat = np.fromstring(R_line.split('=')[1], sep=' ')
    R_matrix = R_flat.reshape((3, 3))

    return C, R_matrix

def ecef_to_geodetic(C):
    transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)
    lon, lat, h = transformer.transform(C[0], C[1], C[2])
    return lon, lat, h

def rotation_matrix_to_rpy(R_cam):
    R_world_to_cam = R_cam.T  # Invert ASP matrix
    r = R.from_matrix(R_world_to_cam)
    roll, pitch, yaw = r.as_euler('xyz', degrees=True)
    return roll, pitch, yaw

def convert_tsai_to_metashape(tsai_files, im_list, output_txt):
    im_list_base = [os.path.basename(x) for x in im_list]

    with open(output_txt, 'w') as f_out:
        # Write header
        f_out.write("# image\tlatitude\tlongitude\theight\troll\tpitch\tyaw\n")

        for tsai_path in tsai_files:
            cam_name = os.path.basename(tsai_path)
            stem = cam_name.replace('_rpc.tsai', '')

            # Match image by base name
            matches = [x for x in im_list_base if os.path.splitext(x)[0] == stem]
            if not matches:
                print(f"Warning: No image match found for {cam_name}")
                continue

            image_name = matches[0]
            try:
                C, R_cam = parse_tsai(tsai_path)
                lat, lon, h = ecef_to_geodetic(C)
                roll, pitch, yaw = rotation_matrix_to_rpy(R_cam)
                f_out.write(f"{image_name}\t{lat:.8f}\t{lon:.8f}\t{h:.3f}\t{roll:.2f}\t{pitch:.2f}\t{yaw:.2f}\n")
            except Exception as e:
                print(f"Error processing {tsai_path}: {e}")

cams_out = os.path.join(cam_gen_fol, 'refined_cameras_list.txt')
convert_tsai_to_metashape(cam_list, im_list, cams_out)

In [ ]:
import os

def asp_to_metashape_gcps(gcp_files, output_txt):
    marker_counter = 0 

    with open(output_txt, 'w') as out:
        # Write header
        out.write("# marker_label\timage\tx_pixel\ty_pixel\tx_coord\ty_coord\tz_coord\n")

        for gcp_file in gcp_files:
            with open(gcp_file, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()

                # ASP format: ID lat lon height image_path col row
                lat = float(parts[1])
                lon = float(parts[2])
                height = float(parts[3])
                image_path = parts[7]
                col = float(parts[8])
                row = float(parts[9])

                marker_label = f"GCP_{marker_counter:004d}" # zero-padded for easy sorting
                image_label = os.path.basename(image_path)

                out.write(f"{marker_label}\t{image_label}\t{col:.2f}\t{row:.2f}\t{lon:.8f}\t{lat:.8f}\t{height:.3f}\n")
                marker_counter += 1

gcp_out = os.path.join(cam_gen_fol, 'gcp_list.txt')
asp_to_metashape_gcps(gcp_list, gcp_out)

In [3]:
C = [-1907965.831, -4439019.286, 4878224.448]
transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)
lon, lat, h = transformer.transform(C[0], C[1], C[2])
print(f"lat: {lat:.8f}, lon: {lon:.8f}, h: {h:.2f}")

lat: 45.45302148, lon: -113.25885394, h: 498707.17
